In [ ]:
import time
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2
import kl_ucb_policy

## Scenario 3
Bras suivant une récompense exponentielle tronquée

### Paramètres du scenario